In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv


**Hiiiiiiiiiiiiiii, going to get intution about the BackPropagation through this Project**

In [2]:
df= pd.read_csv("/kaggle/input/credit-card-customer-churn-prediction/Churn_Modelling.csv")

In [3]:
df.sample(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
8402,8403,15766683,Coombes,549,Germany,Male,36,6,139422.37,1,0,0,83983.39,1
3314,3315,15628767,Hotchin,608,Spain,Female,63,3,139529.93,2,1,1,175696.16,1
5027,5028,15807989,Wall,681,Germany,Male,37,8,73179.34,2,1,1,25292.53,0
6956,6957,15696605,Angelo,571,France,Male,49,4,180614.04,1,0,0,523.00,0
9592,9593,15766066,Nikitina,668,Germany,Female,28,1,124511.01,1,0,0,114258.18,0
8254,8255,15664311,Yang,637,Germany,Male,28,3,123675.69,1,1,1,166458.41,0
2607,2608,15814209,Capon,814,France,Male,31,1,118870.92,1,1,0,101704.19,0
616,617,15719352,Davidson,754,Spain,Male,39,6,170184.99,2,1,0,89593.26,0
8984,8985,15726289,Cawood,645,France,Male,25,0,174400.36,1,1,0,42669.37,0
3889,3890,15756150,Alexander,418,France,Female,39,2,0.00,2,0,0,9041.71,0


In [4]:
df.info() # ok, no null values, nice

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df['Geography'].value_counts() # hmm only 3 countries

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [6]:
df['Exited'].value_counts() # not a greate sign, data

Exited
0    7963
1    2037
Name: count, dtype: int64

In [7]:
# dropping columns which are of no use in this analysis
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1,inplace=True)


In [8]:
df.head() # ok, took longer than expected to drop columns, was making a mistake of nor including axis=1 => By default axis=0 (row)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
df.shape


(10000, 11)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [11]:
# here we need to trasform (One-hot-encode) the categorical data into numerical for analysis
df = pd.get_dummies(df,columns=['Geography','Gender'], drop_first=True )

# see here what we did is, converted them in mumerical also, we did drop_first = true => which means one columns drops, for eq. Gender have 2 Categories so one is represented as 1 and other as 0;

In [12]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,True,False


Here we got another problem, the values are not in one scale which will effect on converging the Data due to the randoms weights between neurons

In [13]:
X = df.drop(columns=['Exited'])
y= df['Exited']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [14]:
y


0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
X_train_scaled # uff it looks scary, it transforms all the 10k rows at one scale 


array([[ 0.35649971, -0.6557859 ,  0.34567966, ..., -0.57946723,
        -0.57638802,  0.91324755],
       [-0.20389777,  0.29493847, -0.3483691 , ...,  1.72572313,
        -0.57638802,  0.91324755],
       [-0.96147213, -1.41636539, -0.69539349, ..., -0.57946723,
         1.73494238,  0.91324755],
       ...,
       [ 0.86500853, -0.08535128, -1.38944225, ..., -0.57946723,
        -0.57638802, -1.09499335],
       [ 0.15932282,  0.3900109 ,  1.03972843, ..., -0.57946723,
        -0.57638802,  0.91324755],
       [ 0.47065475,  1.15059039, -1.38944225, ...,  1.72572313,
        -0.57638802,  0.91324755]])

In [17]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Input

2025-07-04 06:21:19.649449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751610079.866211      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751610079.933266      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Now we are going to Build Sequential Model

In [18]:
model = Sequential()

# for now we will have one input layer, one hidden layer and one output layer
# will put 3 perceptron in hidden layer


# Add input layer + hidden layer (3 neurons, sigmoid)
model.add(Input(shape=(11,)))
model.add(Dense(3, activation='sigmoid'))

# Add output layer (1 neuron, sigmoid)
model.add(Dense(1, activation='sigmoid'))

2025-07-04 06:21:34.012098: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 3)                   │              36 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               4 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 40 (160.00 B)

 Trainable params: 40 (160.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(loss='binary_crossentropy',optimizer='adam')

In [21]:
model.fit(X_train_scaled,y_train,epochs=30)

Epoch 1/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.5387
Epoch 2/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5001
Epoch 3/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4859
Epoch 4/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4649
Epoch 5/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4617
Epoch 6/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4487
Epoch 7/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4550
Epoch 8/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4398
Epoch 9/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4305
Epoch 10/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4384
Epoch 11/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4350
Epoch 12/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4376
Epoch 13/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4337
Epoch 14/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4298
Epoch 15/30
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - lo

In [22]:
model.layers[0].get_weights() # layer[0] points to the input layer and the output weights shows the index wise the weights of the connection between each neuron also there is an second arry which shows the weight of bias


[array([[ 0.257357  ,  0.13299432,  0.00521548],
        [-3.1536348 , -0.44080645, -1.8961362 ],
        [ 0.15799595,  0.16171862, -0.09794328],
        [ 0.31672597, -0.526903  , -0.19824146],
        [ 1.1394908 , -0.19875567, -0.06523695],
        [-0.04155062,  0.11241775, -0.11895514],
        [-0.40728074,  1.2767601 ,  0.5547078 ],
        [ 0.18836723, -0.4362929 ,  0.34430194],
        [-0.30123046, -0.98662335, -0.26428673],
        [-0.42982143,  0.10891613, -0.08096061],
        [ 0.13580814,  0.5659893 ,  0.40504628]], dtype=float32),
 array([0.54905283, 0.26232034, 0.10146227], dtype=float32)]

In [23]:
model.layers[1].get_weights()

[array([[-1.07753  ],
        [-1.5560627],
        [-1.3278164]], dtype=float32),
 array([0.5435857], dtype=float32)]

In [24]:
model.predict(X_test_scaled) # here we go, got the predict value, but it is not 0 and 1, beacuse we used sigmoid function which put these as a probabilities

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


array([[0.18517734],
       [0.08839341],
       [0.3709814 ],
       ...,
       [0.3842225 ],
       [0.05126937],
       [0.21359041]], dtype=float32)

In [25]:
y_log = model.predict(X_test_scaled)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [26]:
y_pred = np.where(y_log>0.5, 1,0)

In [27]:
y_pred # see output changes

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.822

Got the accuracy of 83.25%, hmmm still better than my 10th percentage, but let me improve it

steps to improve
1. increase the number of epochs
2. Hidden layers activation function, relu performs better than the sigmoid
3. Increse the nodes of hidden layer
4. or can increase the number of hidden layers


there is something val_accuracy which splits the X_train also => used to check the overfitting case
but if we do more, overfitting hoggaaa